<a href="https://colab.research.google.com/github/nilanahar/891_NLP/blob/main/891_NLP_Credit_Risk_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np


In [ ]:
# define my google drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# Replace 'train.csv' and 'test.csv' with your actual file names.
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/891/Data/kiva_train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/891/Data/kiva_test.csv')

# Now you can work with the dataframes
df_train.head()

,id,country,en,gender,loan_amount,nonpayment,sector,status
0,1,Ecuador,<h4>Business Description</h4> \r\n <p> Don Mau...,M,825,lender,Food,1
1,2,Dominican Republic,Rosa Iris is a brilliant entrepreneur who sell...,F,450,partner,Retail,0
2,3,Kenya,Sirote is married with six children. Two of he...,F,600,lender,Agriculture,1
3,4,Kenya,David Mwangi Kimani is 33 years old and marri...,M,650,lender,Food,1
4,5,Dominican Republic,Nilda is a very persistent woman who has learn...,F,325,partner,Food,0


In [ ]:
from sklearn.model_selection import train_test_split

X = df_train.drop('status', axis=1)
y = df_train['status']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=20170406) # Adjust test_size and random_state as needed.

print("Training data shape:", X_train.shape, y_train.shape)
print("Validation data shape:", X_val.shape, y_val.shape)


Training data shape: (4363, 7) (4363,)
Validation data shape: (1091, 7) (1091,)


In [ ]:
from sklearn.preprocessing import OneHotEncoder


def ohe(X_train, X_val, col):
  # Initialize OneHotEncoder
  ohe = OneHotEncoder(sparse_output=False)

  # Fit and transform the specified column in df_train, and only transform in df_test and df_valid
  X_train_ohe = ohe.fit_transform(X_train[[col]])
  X_val_ohe = ohe.transform(X_val[[col]])

  # Convert the arrays back to DataFrames and preserve the indexes
  X_train_ohe_df = pd.DataFrame(X_train_ohe, columns = ohe.get_feature_names_out([col]), index=X_train.index)
  X_val_ohe_df = pd.DataFrame(X_val_ohe, columns = ohe.get_feature_names_out([col]), index=X_val.index)

  # Concatenate the encoded features back into the original DataFrames, preserving the original indexes
  X_train = pd.concat([X_train, X_train_ohe_df], axis=1)
  X_val = pd.concat([X_val, X_val_ohe_df], axis=1)

  # Drop the original 'answer_type' column as it's now encoded (optional)
  X_train.drop(columns=[col], inplace=True)
  X_val.drop(columns=[col], inplace=True)
  return X_train, X_val

In [ ]:
X_train, X_val = ohe(X_train, X_val, 'country')
X_train, X_val = ohe(X_train, X_val, 'gender')
X_train, X_val = ohe(X_train, X_val, 'nonpayment')
X_train, X_val = ohe(X_train, X_val, 'sector')

In [ ]:
X_train.head()

,id,en,loan_amount,country_Dominican Republic,country_Ecuador,country_Kenya,gender_F,gender_M,nonpayment_lender,nonpayment_partner,...,sector_Entertainment,sector_Food,sector_Health,sector_Housing,sector_Manufacturing,sector_Personal Use,sector_Retail,sector_Services,sector_Transportation,sector_Wholesale
3687,3688,Grace Achieng Ochieng is 38 years old. She is ...,650,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3594,3595,At 75 Mama Anna Mukute is our most senior clie...,150,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2809,2810,Antoinne is a 40 year old Haitian woman who li...,450,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1966,1967,The Communal Bank <em>Las Caleras</em> is loca...,275,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1060,1061,Maria has three years of experience as a selle...,375,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming 'loan_amount' is the column you want to scale
scaler = StandardScaler()

# Fit the scaler on the training data
X_train['loan_amount'] = scaler.fit_transform(X_train[['loan_amount']])

# Transform the validation data using the same scaler
X_val['loan_amount'] = scaler.transform(X_val[['loan_amount']])

X_train.head()

,id,en,loan_amount,country_Dominican Republic,country_Ecuador,country_Kenya,gender_F,gender_M,nonpayment_lender,nonpayment_partner,...,sector_Entertainment,sector_Food,sector_Health,sector_Housing,sector_Manufacturing,sector_Personal Use,sector_Retail,sector_Services,sector_Transportation,sector_Wholesale
3687,3688,Grace Achieng Ochieng is 38 years old. She is ...,-0.125906,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3594,3595,At 75 Mama Anna Mukute is our most senior clie...,-0.960788,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2809,2810,Antoinne is a 40 year old Haitian woman who li...,-0.459859,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1966,1967,The Communal Bank <em>Las Caleras</em> is loca...,-0.752068,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1060,1061,Maria has three years of experience as a selle...,-0.585091,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# Assuming 'en' is the column you want to vectorize using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'en' column in X_train
X_train_en_tfidf = tfidf_vectorizer.fit_transform(X_train['en'])

# Transform the 'en' column in X_val using the same vectorizer
X_val_en_tfidf = tfidf_vectorizer.transform(X_val['en'])

# Convert the sparse matrices to dense arrays if needed
X_train_en_tfidf = X_train_en_tfidf.toarray()
X_val_en_tfidf = X_val_en_tfidf.toarray()

# Create DataFrames from the vectorized data, preserving the original indexes
X_train_en_tfidf_df = pd.DataFrame(X_train_en_tfidf, columns=tfidf_vectorizer.get_feature_names_out(), index=X_train.index)
X_val_en_tfidf_df = pd.DataFrame(X_val_en_tfidf, columns=tfidf_vectorizer.get_feature_names_out(), index=X_val.index)

# Concatenate the TF-IDF features back into the original DataFrames
X_train = pd.concat([X_train, X_train_en_tfidf_df], axis=1)
X_val = pd.concat([X_val, X_val_en_tfidf_df], axis=1)

# Drop the original 'en' column
X_train.drop(columns=['en'], inplace=True)
X_val.drop(columns=['en'], inplace=True)

X_train.head()

,id,loan_amount,country_Dominican Republic,country_Ecuador,country_Kenya,gender_F,gender_M,nonpayment_lender,nonpayment_partner,sector_Agriculture,...,zones,zoraida,zoraidas,zuleah,zuleida,zuleika,zuleya,zulnilda,zumba,zunilda
3687,3688,-0.125906,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3594,3595,-0.960788,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2809,2810,-0.459859,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1966,1967,-0.752068,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1060,1061,-0.585091,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# prompt: fit a simple xgboost to this data

!pip install xgboost

import xgboost as xgb

# Create DMatrix objects for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set parameters for XGBoost
params = {
    'objective': 'binary:logistic',  # For binary classification
    'eval_metric': 'logloss',        # Evaluation metric
    'eta': 0.1,                      # Learning rate
    'max_depth': 6,                  # Maximum depth of trees
    'subsample': 0.8,                # Subsample ratio of the training instances
    'colsample_bytree': 0.8,         # Subsample ratio of columns when constructing each tree
    'seed': 42                        # Random seed for reproducibility
}

# Train the XGBoost model
num_rounds = 100  # Number of boosting rounds
model = xgb.train(params, dtrain, num_rounds, evals=[(dval, 'eval')])

AttributeError: 'DataFrame' object has no attribute 'dtype'